# Westeros Tutorial - Introducing Sankey diagrams

Sankey diagrams are a useful technique to visualize energy flow accounts.

This tutorial introduces the sankey feature provided by the ``pyam`` packages.


**Pre-requisites**
- You have the *MESSAGEix* framework installed and working
  In particular, you should have installed ``message_ix``, ``pyam``, and ``plotly``.
- Complete tutorial Part 1 (``westeros_baseline.ipynb``) and Introducing Reporting (``westeros_report.ipynb``).

We start as usual by connecting to a database and loading a scenario. Note that we do not `clone()` the scenario here because we do not intend to make any changes to it. 

In [18]:
import ixmp

from message_ix import Scenario

mp = ixmp.Platform()
scenario = Scenario(mp, model="Westeros Electrified", scenario="baseline")

# Ensure the scenario has a solution
if not scenario.has_solution():
    scenario.solve(quiet=True)

Next, we create the `Reporter` object. Since ``"-"`` is a reserved character in the unit-handling [pint](https://github.com/hgrecco/pint), we need to replace it by ``""``.

In [19]:
from message_ix.report import Reporter

rep = Reporter.from_scenario(scenario)

rep.configure(units={"replace": {"-": ""}})

This `Reporter` already includes everything we need to construct the `pyam.IamDataFrame` required for plotting Sankey diagrams! In other words, it includes the input and output flows in the IAMC format (`in::pyam` and `out::pyam`, respectively). We can start the calculation manually:

In [ ]:
from genno.operator import concat

pyam_out = rep.get("out::pyam")
pyam_in = rep.get("in::pyam")

concat(pyam_out, pyam_in)

# Please note: if you don't use the convenience function below, you need to store the
# result of concat(pyam_out, pyam_in) as df here!

Or we can use a built-in convenience function. This will also add the calculation to the `Reporter`, so the same calculation would not need to be repeated if it's used anywhere else, saving us time and memory.

In [5]:
rep.add_sankey()

The resulting `pyam.IamDataFrame` is accessible through the key `message::sankey`:


In [6]:
df = rep.get("message::sankey")

Now, we can use the utility function `map_for_sankey(iam_df, year, region, exclude=[])` to create the mapping required for the `figures.sankey()` function of the `pyam` package. Each Sankey diagram will depict one year and region, which we have to provide to the function. In some models it might be necessary to exclude variables and flows to get meaningful Sankey diagrams; for this, you can use `exclude` as detailed below. But let´s try with all!

In [7]:
from message_ix.util.sankey import map_for_sankey

mapping = map_for_sankey(df, year=700, region="Westeros")

The pyam function `pyam.figures.sankey()`returns a `plotly` figure object of our desired Sankey diagram that can be further modified. However, it can currently only handle data for single years, so we need to ensure that the input data we provide is filtered for the same year we filtered for above.  

Finally, we can plot it as an interactive diagram!

In [ ]:
from pyam.figures import sankey

fig = sankey(df=df.filter(year=700), mapping=mapping)
fig.show()

With hundreds of variables, you can imagine this diagram getting crowded! We can use the `exclude` parameter of `map_for_sankey()` to exclude variables we are not interested in:

In [14]:
mapping_without_wind_ppl_standard = map_for_sankey(
    df,
    year=700,
    region="Westeros",
    exclude=["wind_ppl|standard"],
)

Then, we can display the figure as before:

In [ ]:
fig = sankey(df=df.filter(year=700), mapping=mapping_without_wind_ppl_standard)
fig.show()

You can pick any variable for this, even if it's in the middle of another flow! And for this scenario, you can pick other years, too:

In [ ]:
mapping_without_final_electricity = map_for_sankey(
    df, year=720, region="Westeros", exclude=["final|electricity"]
)
fig = sankey(df=df.filter(year=720), mapping=mapping_without_final_electricity)
fig.show()

And lastly, as always, please do not forget to close the database ;-) 

In [17]:
mp.close_db()